# Team 7 - Garbage DumPY
## Demonstration of Basic Forward and Inverse Kinematics
### MANE 4120/6120 - Robotics 1 - Fall 2023

1. Import RPI Robotics Toolbox, Yahboom Jetson DOFBOT Code, etc, and Initialize Robot Object

In [23]:
import general_robotics_toolbox.general_robotics_toolbox as grt
import general_robotics_toolbox.general_robotics_toolbox_invkin as grt_ik
import Arm, Dofbot
from Arm_Lib import Arm_Device
import time
import numpy as np
import math
import path_generation
import move_read

# Define all the joint lengths [m]
l0 = 61 * 10**-3
l1 = 43.5 * 10**-3
l2 = 82.85 * 10**-3
l3 = 82.85 * 10**-3
l4 = 73.85 * 10**-3
l5 = 54.57 * 10**-3


# define the unit vectors
ex = np.array([1, 0, 0])
ey = np.array([0, 1, 0])
ez = np.array([0, 0, 1])


# Define the position vectors from i-1 -> i
P01 = (l0 + l1) * ez
P12 = np.zeros(3)
P23 = l2 * ex
P34 = -1*l3 * ez
P45 = np.zeros(3)
P5T = -1*(l4 + l5) * ex

# define the class inputs: rotation axes (H), position vectors (P), and joint_type
H = np.array([ez, -1*ey, -1*ey, -1*ey, -1*ex]).T
P = np.array([P01, P12, P23, P34, P45, P5T]).T
joint_type = [0,0,0,0,0]

 
# define the Robot object
robot = grt.Robot(H, P, joint_type)

# initialize servo arm
Arm = Arm_Device()

2. Initial Configuration

In [24]:
# Set the bot to be vertical
#Arm.Arm_serial_servo_write6(0,90,10,45,90,15,1100)
start = np.array([90,90,90,90,90,15])
move_read.move_bot(start)
# THE SERVO ANGLE (the 6th variable) CAN NOT BE ZERO OR IT WILL BREAK THE SERVO
# It must be between 5 and 180

# initialize joint angles, radians
q0 = np.array([90,70,5,55,90])*math.pi/180

print("Initial Joints:\n",q0)
# and compute path
p = grt.fwdkin(robot,q0)
print("Expected Transform:\n",p)

#End position from FK
end1 = np.array([90,70,5,55,90,15])
#move_read.move_bot(end)

#Generating path
q_des = path_generation.gen_joint_space_path(start, end1, 5)

#Moving arm through path
path_generation.path_follow(q_des,2,5)
    
print(move_read.read_joints())

#Grabbing block
Arm.Arm_serial_servo_write(6, 155, 400)
time.sleep(0.4)
print('grabbed')

# desired rotation matrix
Rf = np.array([[-0.8192,0.5736,0],[0,0,1],[.5736,.8192,0]])
# desired position matrix
Pf = [0.1868,0,0.1281]
# make desired transformation
Tf = grt.Transform(Rf,Pf)
# and calculate inverse kinematics
#DO NOT OVERWRITE QF (DUMPER LOCATION)
max_steps=200
qf = grt_ik.iterative_invkin(robot,Tf,q0,max_steps)
while(qf[0]!=True): #if the inverse kinematics does not converge
    max_steps*=2
    qf = grt_ik.iterative_invkin(robot,Tf,q0,max_steps)
print("Necessary Joint Angles:\n",np.array(qf[1][0]))


#Path Following

start_garb_place = move_read.read_joints()
end_garb_place = np.append(np.array(qf[1][0])*180/math.pi, move_read.read_joints()[5])

steps = 10
t = 1.5
kp = .3
ki = .3
kd = .01

q_des = path_generation.gen_joint_space_path(start_garb_place, end_garb_place, steps)
print(q_des)

error = path_generation.pid_path_follow(q_des, t, steps, kp, ki, kd)

print(error)

print(move_read.read_joints())

#Releasing block
Arm.Arm_serial_servo_write(6, 125, 400)
time.sleep(0.4)
print('dropped')

Initial Joints:
 [1.57079633 1.22173048 0.08726646 0.95993109 1.57079633]
Expected Transform:
 R = [[-0.     -0.     -1.    ]
     [-0.6428  0.766   0.    ]
     [ 0.766   0.6428 -0.    ]]
p = [0.     0.1909 0.0625]


[89. 70.  5. 56. 90. 14.]
grabbed
Necessary Joint Angles:
 [1.45405040e-04 1.57056825e+00 1.75056200e-01 7.85082530e-01
 1.57069850e+00]
[array([ 89.,  70.,   5.,  56.,  90., 142.]), array([ 80.10083311,  71.99869323,   5.50299815,  54.89819156,
        89.99943951, 142.        ]), array([ 71.20166622,  73.99738645,   6.00599629,  53.79638311,
        89.99887902, 142.        ]), array([ 62.30249933,  75.99607968,   6.50899444,  52.69457467,
        89.99831853, 142.        ]), array([ 53.40333244,  77.9947729 ,   7.01199258,  51.59276622,
        89.99775804, 142.        ]), array([ 44.50416555,  79.99346613,   7.51499073,  50.49095778,
        89.99719754, 142.        ]), array([ 35.60499866,  81.99215935,   8.01798887,  49.38914934,
        89.99663705, 142.        ]),

Block 2

In [25]:
# initialize joint angles, radians
q2 = np.array([35,80,10,35,90])

print("Initial Joints:\n",q2)
# and compute path
p = grt.fwdkin(robot,q2)
print("Expected Transform:\n",p)

#End position from FK
start2 = move_read.read_joints()
end2 = np.array([40,80,5,35,90,15])

#Generating path
q_des2 = path_generation.gen_joint_space_path(start2, end2, 5)

#Moving arm through path
path_generation.path_follow(q_des2,2,5)
    
print(move_read.read_joints())

#Grabbing block
Arm.Arm_serial_servo_write(6, 155, 400)
time.sleep(0.4)
print('grabbed')

#Path Following

start_garb_place = move_read.read_joints()
end_garb_place = np.append(np.array(qf[1][0])*180/math.pi, move_read.read_joints()[5])

steps = 10
t = 1.5
kp = .3
ki = .3
kd = .01

q_des = path_generation.gen_joint_space_path(start_garb_place, end_garb_place, steps)
print(q_des)

error = path_generation.pid_path_follow(q_des, t, steps, kp, ki, kd)

print(error)

print(move_read.read_joints())

#Releasing block
Arm.Arm_serial_servo_write(6, 125, 400)
time.sleep(0.4)
print('dropped')

Initial Joints:
 [35 80 10 35 90]
Expected Transform:
 R = [[-0.7119  0.3058  0.6322]
     [-0.3373  0.6407 -0.6897]
     [-0.616  -0.7042 -0.353 ]]
p = [0.0327 0.0155 0.1384]


[39. 81.  6. 36. 90. 15.]
grabbed
[array([ 39.,  80.,   5.,  36.,  90., 142.]), array([ 35.10083311,  80.99869323,   5.50299815,  36.89819156,
        89.99943951, 142.        ]), array([ 31.20166622,  81.99738645,   6.00599629,  37.79638311,
        89.99887902, 142.        ]), array([ 27.30249933,  82.99607968,   6.50899444,  38.69457467,
        89.99831853, 142.        ]), array([ 23.40333244,  83.9947729 ,   7.01199258,  39.59276622,
        89.99775804, 142.        ]), array([ 19.50416555,  84.99346613,   7.51499073,  40.49095778,
        89.99719754, 142.        ]), array([ 15.60499866,  85.99215935,   8.01798887,  41.38914934,
        89.99663705, 142.        ]), array([ 11.70583177,  86.99085258,   8.52098702,  42.28734089,
        89.99607656, 142.        ]), array([  7.80666488,  87.98954581,   9.023

BLOCK 3

In [26]:
# initialize joint angles, radians
q3 = np.array([135,72,9,40,90,15])

print("Initial Joints:\n",q3)
# and compute path
p = grt.fwdkin(robot,q3)
print("Expected Transform:\n",p)

#End position from FK
start3 = move_read.read_joints()
end3 = q3

#Generating path
q_des3 = path_generation.gen_joint_space_path(start3, end3, 5)

#Moving arm through path
path_generation.path_follow(q_des3,2,5)
    
print(move_read.read_joints())

#Grabbing block
Arm.Arm_serial_servo_write(6, 155, 400)
time.sleep(0.4)
print('grabbed')

#Path Following

start_garb_place = move_read.read_joints()
end_garb_place = np.append(np.array(qf[1][0])*180/math.pi, move_read.read_joints()[5])

steps = 10
t = 1.5
kp = .3
ki = .3
kd = .01

q_des = path_generation.gen_joint_space_path(start_garb_place, end_garb_place, steps)
print(q_des)

error = path_generation.pid_path_follow(q_des, t, steps, kp, ki, kd)

print(error)

print(move_read.read_joints())

#Releasing block
Arm.Arm_serial_servo_write(6, 125, 400)
time.sleep(0.4)
print('dropped')

Initial Joints:
 [135  72   9  40  90  15]
Expected Transform:
 R = [[ 0.0485 -0.8498 -0.5248]
     [-0.0043  0.5252 -0.851 ]
     [ 0.9988  0.0435  0.0218]]
p = [ 0.1256 -0.0111 -0.0671]


[134.  90.   9.  41.  90.  14.]
grabbed
[array([134.,  72.,   9.,  41.,  90., 142.]), array([120.60083311,  73.79869323,   9.10299815,  41.39819156,
        89.99943951, 142.        ]), array([107.20166622,  75.59738645,   9.20599629,  41.79638311,
        89.99887902, 142.        ]), array([ 93.80249933,  77.39607968,   9.30899444,  42.19457467,
        89.99831853, 142.        ]), array([ 80.40333244,  79.1947729 ,   9.41199258,  42.59276622,
        89.99775804, 142.        ]), array([ 67.00416555,  80.99346613,   9.51499073,  42.99095778,
        89.99719754, 142.        ]), array([ 53.60499866,  82.79215935,   9.61798887,  43.38914934,
        89.99663705, 142.        ]), array([ 40.20583177,  84.59085258,   9.72098702,  43.78734089,
        89.99607656, 142.        ]), array([ 26.80666488,  86

BLOCK 4

In [27]:
# initialize joint angles, radians
q4 = np.array([65,25,60,60,90,15])

print("Initial Joints:\n",q4)
# and compute path
p = grt.fwdkin(robot,q4)
print("Expected Transform:\n",p)

#End position from FK
start4 = move_read.read_joints()
end4 = np.array([62,25,65,60,90,15])

#Generating path
q_des4 = path_generation.gen_joint_space_path(start4, end4, 5)

#Moving arm through path
path_generation.path_follow(q_des4,2,5)
    
print(move_read.read_joints())

#Grabbing block
Arm.Arm_serial_servo_write(6, 155, 400)
time.sleep(0.4)
print('grabbed')

#Path Following

start_garb_place = move_read.read_joints()
end_garb_place = np.append(np.array(qf[1][0])*180/math.pi, move_read.read_joints()[5])

steps = 10
t = 1.5
kp = .3
ki = .3
kd = .01

q_des = path_generation.gen_joint_space_path(start_garb_place, end_garb_place, steps)
print(q_des)

error = path_generation.pid_path_follow(q_des, t, steps, kp, ki, kd)

print(error)

print(move_read.read_joints())

#Releasing block
Arm.Arm_serial_servo_write(6, 125, 400)
time.sleep(0.4)
print('dropped')

Initial Joints:
 [65 25 60 60 90 15]
Expected Transform:
 R = [[-0.4971  0.1353 -0.8571]
     [ 0.7308  0.5978 -0.3295]
     [ 0.4677 -0.7902 -0.396 ]]
p = [ 0.0259 -0.038   0.115 ]


[61. 24. 90. 60. 90. 14.]
grabbed
[array([ 61.,  24.,  64.,  60.,  90., 142.]), array([ 54.90083311,  30.59869323,  58.60299815,  58.49819156,
        89.99943951, 142.        ]), array([ 48.80166622,  37.19738645,  53.20599629,  56.99638311,
        89.99887902, 142.        ]), array([ 42.70249933,  43.79607968,  47.80899444,  55.49457467,
        89.99831853, 142.        ]), array([ 36.60333244,  50.3947729 ,  42.41199258,  53.99276622,
        89.99775804, 142.        ]), array([ 30.50416555,  56.99346613,  37.01499073,  52.49095778,
        89.99719754, 142.        ]), array([ 24.40499866,  63.59215935,  31.61798887,  50.98914934,
        89.99663705, 142.        ]), array([ 18.30583177,  70.19085258,  26.22098702,  49.48734089,
        89.99607656, 142.        ]), array([ 12.20666488,  76.78954581,  

BLOCK 5

In [28]:
# initialize joint angles, radians
q5 = np.array([115,30,60,60,90,15])

print("Initial Joints:\n",q5)
# and compute path
p = grt.fwdkin(robot,q5)
print("Expected Transform:\n",p)

#End position from FK
start5 = move_read.read_joints()
end5 = q5

#Generating path
q_des5 = path_generation.gen_joint_space_path(start5, end5, 5)

#Moving arm through path
path_generation.path_follow(q_des5,2,5)
    
print(move_read.read_joints())

#Grabbing block
Arm.Arm_serial_servo_write(6, 155, 400)
time.sleep(0.4)
print('grabbed')

#Path Following

start_garb_place = move_read.read_joints()
end_garb_place = np.append(np.array(qf[1][0])*180/math.pi, move_read.read_joints()[5])

steps = 10
t = 1.5
kp = .1
ki = .2
kd = .01

q_des = path_generation.gen_joint_space_path(start_garb_place, end_garb_place, steps)
print(q_des)

error = path_generation.pid_path_follow(q_des, t, steps, kp, ki, kd)

print(error)

print(move_read.read_joints())

#Releasing block
Arm.Arm_serial_servo_write(6, 125, 400)
time.sleep(0.4)
print('dropped')

Initial Joints:
 [115  30  60  60  90  15]
Expected Transform:
 R = [[-0.2278  0.6318 -0.7409]
     [ 0.6611 -0.4582 -0.5941]
     [-0.7149 -0.6251 -0.3133]]
p = [ 0.001  -0.0028  0.1516]


[114.  30.  58.  59.  90.  14.]
grabbed
[array([114.,  29.,  59.,  60.,  90., 142.]), array([102.60083311,  35.09869323,  54.10299815,  58.49819156,
        89.99943951, 142.        ]), array([ 91.20166622,  41.19738645,  49.20599629,  56.99638311,
        89.99887902, 142.        ]), array([ 79.80249933,  47.29607968,  44.30899444,  55.49457467,
        89.99831853, 142.        ]), array([ 68.40333244,  53.3947729 ,  39.41199258,  53.99276622,
        89.99775804, 142.        ]), array([ 57.00416555,  59.49346613,  34.51499073,  52.49095778,
        89.99719754, 142.        ]), array([ 45.60499866,  65.59215935,  29.61798887,  50.98914934,
        89.99663705, 142.        ]), array([ 34.20583177,  71.69085258,  24.72098702,  49.48734089,
        89.99607656, 142.        ]), array([ 22.80666488,  77